In [ ]:
%env DSP_CACHEBOOL=false

In [ ]:
%run ./module-00-data-prep.ipynb
train = train
val = val
test = test
len(train), len(val), len(test)

In [ ]:
import dspy

MODEL = "qwen2.5:1.5b"

lm = dspy.OllamaLocal(
    model=MODEL,
)
dspy.configure(lm=lm)

dspy.settings.configure(lm=lm)

In [ ]:
from sms_classifier import SMSClassifier

sms_classifier = SMSClassifier(lm)

In [ ]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch
from evaluation_helpers import validate_answer

config = dict(
    max_bootstrapped_demos=4,
    max_labeled_demos=8,
    num_candidate_programs=16,
    num_threads=1,
    max_errors=10,
)

teleprompter = BootstrapFewShotWithRandomSearch(metric=validate_answer, **config)
optimized_program = teleprompter.compile(sms_classifier, trainset=train[:100])

### Evaluation

In [ ]:
from datetime import datetime

from notebooks.cybersecurity.langfuse_extensions import EvaluateWithLangfuse

session = f"Run-{MODEL}-optim-{int(datetime.now().timestamp())}"
print(session)
evaluator = EvaluateWithLangfuse(
    devset=test, num_threads=1, display_progress=True, run_id=session
)
evaluator(optimized_program, metric=validate_answer)

### Load previous saved program

In [ ]:
optimized_program.save("programs/sms_classifier-qwen2.5-3b.json")